In [2]:
import pandas as pd
import sqlite3
import plotly.express as px
import streamlit as st

# Charger les données des objets trouvés et des gares depuis la base de données SQLite
conn = sqlite3.connect('../objets_trouves.db')
query_objets_trouves = "SELECT * FROM objets_trouves"
query_temperature = "SELECT * FROM temperature"
df_objets_trouves = pd.read_sql_query(query_objets_trouves, conn)
df_temperature = pd.read_sql_query(query_temperature, conn)

In [3]:
df_objets_trouves['date'] = df_objets_trouves['date'].apply(lambda x: pd.to_datetime(x).date() if pd.notnull(x) else x)
df_temperature['date'] = df_temperature['date'].apply(lambda x: pd.to_datetime(x).date() if pd.notnull(x) else x)

In [4]:
df_objets_trouves=df_objets_trouves.groupby(['date']).count()['gare'].rename('nombre_objets').to_frame().reset_index()

In [5]:
df = pd.merge(df_objets_trouves, df_temperature, on='date')

In [6]:
df

,date,nombre_objets,id,temperature
0,2019-01-01,5,1,7.5
1,2019-01-02,90,2,5.7
2,2019-01-03,107,3,2.4
3,2019-01-04,84,4,1.5
4,2019-01-05,78,5,2.8
...,...,...,...,...
1423,2022-12-27,84,1457,5.5
1424,2022-12-28,67,1458,10.0
1425,2022-12-29,65,1459,9.8
1426,2022-12-30,61,1460,9.6


In [7]:
# Création du scatterplot
fig = px.scatter(df, x='temperature', y='nombre_objets', 
                 title='Relation entre la température et le nombre d\'objets trouvés')
fig.show()